In [8]:
import numpy as np
import os
from soma import aims, aimsalgo
import pandas as pd

In [5]:
directory = '/volatile/jl277509/data/UkBioBank/crops/2mm/CINGULATE/mask/'
side = 'R'

In [10]:
crops_dirs = directory+side+'crops/'
skel_subjects = pd.read_csv(directory+side+'skeleton_subject.csv')

In [31]:
distbottom_list = []
fm = aims.FastMarching()
for i, subject in enumerate(skel_subjects.Subject):
    vol = aims.read(crops_dirs+subject+'_cropped_skeleton.nii.gz')
    vol[vol.np > 30] = 100
    # carte de distance par FastMarching
    # propagation dans le label 100 (squelette) à partir de la graine 30 (bottom), sans graine extérieure (-1 n'existe pas dans l'image)
    dist = fm.doit(vol, 100, 30, -1)
    # convertir en np, modifier le type, puis reconvertir en vol int16
    arr_to_write = dist.np
    outside = np.max(arr_to_write)
    inf_dist=2**15-1
    arr_to_write[arr_to_write==outside]=inf_dist
    arr_to_write = arr_to_write.astype(np.int16)
    arr_to_write = np.array(arr_to_write, order='F')
    vol_to_write = aims.Volume(arr_to_write)
    aims.write(vol_to_write, directory+side+f'distbottom/{subject}_cropped_distbottom.nii.gz')
    distbottom_list.append(arr_to_write)

In [43]:
arr_distbottom_all = np.stack(distbottom_list)

In [44]:
arr_distbottom_all.shape

(21052, 16, 37, 37, 1)

In [54]:
np.random.randint(0,1)

0

In [45]:
np.save(directory+side+'distbottom.npy', arr_distbottom_all)

In [32]:
a = np.load('/volatile/jl277509/data/UkBioBank/crops/2mm/CINGULATE/mask/Rskeleton.npy')

In [33]:
a.shape

(21052, 16, 37, 37, 1)

In [29]:
b = np.load('/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/crops/2mm/CINGULATE/mask/Rskeleton.npy')

In [30]:
b.shape

(341, 16, 37, 37, 1)